In [2]:
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
from numpy import percentile
from numpy.random import randn
import numpy as np
from scipy.stats import norm, stats

In [ ]:
def graphData(year, group="all"):
    pd.read_csv("data/WICAgencies{}ytd/".format(year))
    